In [1]:
import os, sys, shutil
import subprocess
import fnmatch
import numpy as np
import glob
import pandas as pd

## Settings

In [2]:
# animal folder

# file_path = '/media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans'
file_path = '/media/sf_MRIDATA/Gadospin2018/20180903/control/horizontal_scans'


template = '/media/sf_MRIDATA/Gadospin2018/20180903/template/X12P1.nii'

ROI = '/media/sf_MRIDATA/Gadospin2018/20180903/template/ROI_horizontal.nii'

## Convert scans

In [3]:
def runAFNI(inputstring):
    print(inputstring)
    subprocess.run(inputstring,shell=True, check=True)

In [4]:
main_path = os.path.dirname(file_path)
runAFNI(f'Bru2 {main_path}/subject')

Bru2 /media/sf_MRIDATA/Gadospin2018/20180903/test/subject


##  Extract ROIs

In [ ]:
def splitall(path):
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return allparts


# for folderName, subfolders, filenames in os.walk(file_path):
#     for file in filenames:
#         if fnmatch.fnmatch(file, '*.nii'):
#             file_path1 = glob.glob(os.path.join(folderName,file))
#             file_path1 = file_path1[0]
            
#             for horizontal_scan in horizontal_scans:
#                 if fnmatch.fnmatch(file_path1, f'*{horizontal_scan}P1.nii'):
#                     print('horizontal scan')
#                     print('extracting rois from: ', file_path1)
#                     runAFNI(f'3dROIstats -mask {file_path}/{h_ROI} -sigma {file_path1} > {folderName}/ROIdata_h{horizontal_scan}.1D')
#                 if fnmatch.fnmatch(file_path1, f'*{sagittal_scan}P1.nii'):
#                     print('sagittal scan')
#                     runAFNI(f'3dROIstats -mask {file_path}/{s_ROI} {file_path1} > {folderName}/ROIdata_s{horizontal_scan-1}.1D')

for folderName, subfolders, filenames in os.walk(file_path):
    for file in filenames:
        if fnmatch.fnmatch(file, '*.nii'):
            file_path1 = glob.glob(os.path.join(folderName,file))
            file_path1 = file_path1[0] # filename
            ID = splitall(file_path1)[-1][:-6] # scan number only
            print(ID)
#             print('extracting rois from: ', file_path1)
            runAFNI(f"3dAllineate -base {template} -source {file_path1} -prefix {file_path}/{ID}coreg -cost ls -twoblur 1 -final 'wsinc5'")
            runAFNI(f'3dROIstats -mask {ROI} -sigma {file_path1} > {folderName}/ROIdata_{ID}.1D')


X12
3dAllineate -base /media/sf_MRIDATA/Gadospin2018/20180903/template/X12P1.nii -source /media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/X12P1.nii -prefix /media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/X12coreg -cost ls -twoblur 1 -final 'wsinc5'
3dROIstats -mask /media/sf_MRIDATA/Gadospin2018/20180903/template/ROI_horizontal.nii -sigma /media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/X12P1.nii > /media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/ROIdata_X12.1D
X14
3dAllineate -base /media/sf_MRIDATA/Gadospin2018/20180903/template/X12P1.nii -source /media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/X14P1.nii -prefix /media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/X14coreg -cost ls -twoblur 1 -final 'wsinc5'
3dROIstats -mask /media/sf_MRIDATA/Gadospin2018/20180903/template/ROI_horizontal.nii -sigma /media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/X14P1.nii > /media/sf_MRIDATA/Gadospin2018/20180903

## Read all ROI files

In [103]:
allfiles = glob.glob(f'{file_path}/*.1D')

frame = pd.DataFrame()
list_ = []
for file_ in allfiles:
    print(file_)
    df = pd.read_table(file_)
    list_.append(df)
frame = pd.concat(list_)

frame.columns

/media/sf_MRIDATA/Gadospin2018/20180903/test/ROIdata_h12.1D
/media/sf_MRIDATA/Gadospin2018/20180903/test/ROIdata_h14.1D
/media/sf_MRIDATA/Gadospin2018/20180903/test/ROIdata_h17.1D
/media/sf_MRIDATA/Gadospin2018/20180903/test/ROIdata_h19.1D
/media/sf_MRIDATA/Gadospin2018/20180903/test/ROIdata_h21.1D


In [122]:
frame['SNR'] = ( frame['Mean_signalR  ']/frame['Sigma_noiseR'] + frame['Mean_signalL  ']/frame['Sigma_noiseL'] ) / 2

del frame['Sub-brick']

frame.to_excel(f'{file_path}/output.xlsx')
frame

,File,Mean_noiseR,Sigma_noiseR,Mean_noiseL,Sigma_noiseL,Mean_signalR,Sigma_signalR,Mean_signalL,Sigma_signalL,SNR
0,/media/sf_MRIDATA/Gadospin2018/20180903/test/X...,0.000078,0.000020,0.000079,0.000021,0.002109,0.001500,0.001961,0.001149,99.415476
0,/media/sf_MRIDATA/Gadospin2018/20180903/test/X...,0.000080,0.000020,0.000078,0.000020,0.002827,0.001517,0.002879,0.001409,142.650000
0,/media/sf_MRIDATA/Gadospin2018/20180903/test/X...,0.000078,0.000021,0.000078,0.000021,0.003195,0.001455,0.002935,0.001457,145.952381
0,/media/sf_MRIDATA/Gadospin2018/20180903/test/X...,0.000082,0.000021,0.000080,0.000022,0.003287,0.001542,0.002870,0.001422,143.489177
0,/media/sf_MRIDATA/Gadospin2018/20180903/test/X...,0.000085,0.000022,0.000081,0.000021,0.003218,0.001573,0.002592,0.001322,134.850649
